In [1]:
import glob
print(glob.glob("/share/*"))

import torch
import faiss
from torch_cluster import knn_graph
from timeseries.timeseries_transformer import TimeseriesDataloader
from torch_cluster import knn
print(torch.cuda.is_available())
directory_path = '/share/us-stock-dataset/Data/Stocks'


['/share/us-stock-dataset', '/share/Transformers1']
True


In [2]:

stocks_to_load = [
    "AAPL", "TSLA",
    "A", "GOOG", "AMZN", "PYPL", "NVDA", "AMD",
    "NFLX", "MSFT", "INTC", "CSCO", "ADBE", "CRM", "QCOM", "TXN", "AVGO",
    "INTU", "ORCL", "COST", "SBUX", "AMGN", "CHTR", "GILD", "CMCSA", "BKNG",
    "MDLZ", "FISV", "BIIB", "MU", "MCD", "AMAT", "ADP", "ILMN", "ATVI", "ISRG",
    "ADSK", "LRCX", "BIDU", "JD", "REGN", "WBA", "VRTX", "KHC", "WMT", "ZM", "MELI",
    "TMUS", "CTSH", "XLNX", "PCAR", "ALGN", "WDAY", "SIRI", "CTXS", "ADI", "EXC", "LULU",
    "MAR", "KLAC", "PAYX", "EA", "ILMN", "ALXN", "MNST", "BMRN", "EBAY", "CTAS", "VRSK",
    "IDXX", "CDNS", "NXPI", "ASML", "INCY", "KLAC", "MCHP", "SNPS", "SWKS", "VRSN",
    "WDC", "WYNN", "XLNX", "ZBRA", "ZTS", "AEP", "AIG", "ALL", "AXP", "BA", "BAC",
    "BK", "BLK", "C", "CAT", "CL", "COF", "COP", "COST", "CSCO", "CVS", "CVX",
    "DD", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD",
    "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI",
    "KO", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM",
    'BLND', 'BLNG', 'BLNKW', 'BLNGU', 'BLNGW', 'BLNK', 'BLPH', 'BLRX',
    'BLTE', 'BLU', 'BLUA', 'BLUE', 'BLW', 'BLX', 'BLZE', 'BMA', 'BMAC', 'BMAQ', 'BMAQR', 'BMAQU'
]


In [3]:
dataloader = TimeseriesDataloader(directory_path, stocks_to_load, add_diff=False)

Reading and merging CSV files: ['AAPL', 'TSLA', 'A', 'GOOG', 'AMZN', 'PYPL', 'NVDA', 'AMD', 'NFLX', 'MSFT', 'INTC', 'CSCO', 'ADBE', 'CRM', 'QCOM', 'TXN', 'AVGO', 'INTU', 'ORCL', 'COST', 'SBUX', 'AMGN', 'CHTR', 'GILD', 'CMCSA', 'BKNG', 'MDLZ', 'FISV', 'BIIB', 'MU', 'MCD', 'AMAT', 'ADP', 'ILMN', 'ATVI', 'ISRG', 'ADSK', 'LRCX', 'BIDU', 'JD', 'REGN', 'WBA', 'VRTX', 'KHC', 'WMT', 'ZM', 'MELI', 'TMUS', 'CTSH', 'XLNX', 'PCAR', 'ALGN', 'WDAY', 'SIRI', 'CTXS', 'ADI', 'EXC', 'LULU', 'MAR', 'KLAC', 'PAYX', 'EA', 'ILMN', 'ALXN', 'MNST', 'BMRN', 'EBAY', 'CTAS', 'VRSK', 'IDXX', 'CDNS', 'NXPI', 'ASML', 'INCY', 'KLAC', 'MCHP', 'SNPS', 'SWKS', 'VRSN', 'WDC', 'WYNN', 'XLNX', 'ZBRA', 'ZTS', 'AEP', 'AIG', 'ALL', 'AXP', 'BA', 'BAC', 'BK', 'BLK', 'C', 'CAT', 'CL', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD'

In [4]:
data=dataloader.get_data().transpose(0, 1).cuda()
data.shape

torch.Size([145, 5283])

In [18]:
training_data=data.unsqueeze(0)[:,0:16,:]
print(training_data.shape)

torch.Size([1, 16, 5283])


In [19]:
training_data = torch.diff(training_data, dim=2)


In [20]:
training_data.std()

tensor(1.5572, device='cuda:0')

In [21]:
def min_max_scaling(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())


In [22]:
training_data = min_max_scaling(training_data)
training_data.std()

tensor(0.0146, device='cuda:0')

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size=16, dim=256, emb_dim=128, kernel_size1=32, kernel_size2=8, dropout=0.1):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=input_size*emb_dim, kernel_size=kernel_size1, stride=1, padding=0), #, groups=input_size
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=input_size*emb_dim, out_channels=input_size, kernel_size=kernel_size1, stride=1, padding=0), #, groups=input_size
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the model, loss function, and optimizer
model = Autoencoder().cuda()
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(training_data)
    
    # Compute the loss
    loss = loss_function(outputs, training_data)
    
    # Zero the gradients
    optimizer.zero_grad()
    
    # Backward pass
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {torch.sqrt(loss)}")


Epoch [1/50000], Loss: 0.9169793128967285
Epoch [2/50000], Loss: 3.9939181804656982
Epoch [3/50000], Loss: 1.2882394790649414
Epoch [4/50000], Loss: 0.4653642177581787
Epoch [5/50000], Loss: 0.46538034081459045
Epoch [6/50000], Loss: 0.4653835594654083
Epoch [7/50000], Loss: 0.46538445353507996
Epoch [8/50000], Loss: 0.4653871953487396
Epoch [9/50000], Loss: 0.4653871953487396
Epoch [10/50000], Loss: 0.4653871953487396
Epoch [11/50000], Loss: 0.4653871953487396
Epoch [12/50000], Loss: 0.4653871953487396
Epoch [13/50000], Loss: 0.4653871953487396
Epoch [14/50000], Loss: 0.4653871953487396
Epoch [15/50000], Loss: 0.4653871953487396
Epoch [16/50000], Loss: 0.4653871953487396
Epoch [17/50000], Loss: 0.4653871953487396
Epoch [18/50000], Loss: 0.4653871953487396
Epoch [19/50000], Loss: 0.4653871953487396
Epoch [20/50000], Loss: 0.4653871953487396
Epoch [21/50000], Loss: 0.4653871953487396
Epoch [22/50000], Loss: 0.4653871953487396
Epoch [23/50000], Loss: 0.4653871953487396
Epoch [24/50000], 

KeyboardInterrupt: 

In [ ]:
emb_dim = 256

import matplotlib.pyplot as plt

# Initialize the model
model = Autoencoder().cuda()

# Extract weights from the first convolutional layer of the encoder
weights = model.encoder[0].weight.data.cpu().numpy()

# Plot the filters
plt.figure(figsize=(200, 100))

print(weights.shape)

for i, filter in enumerate(weights[0:10]):
    print(i)
    plt.subplot(10, emb_dim, i+1)  # Assuming you want to visualize 10 filters in a row
    plt.plot(filter[0])  # Plotting the filter for the first input channel
    plt.axis('off')

plt.show()
